In [ ]:
# from panda3d.core import Point3
# from panda3d.bullet import BulletWorld, BulletBoxShape, BulletRigidBodyNode
# from direct.showbase.ShowBase import ShowBase

# class MyApp(ShowBase):
#     def __init__(self):
#         ShowBase.__init__(self)
        
#         # 初始化 Bullet 物理世界
#         self.bullet_world = BulletWorld()
#         self.bullet_world.setGravity((0, 0, -9.81))  # 设置重力

#         # 加载模型
#         self.model = self.loader.loadModel("models/box")
#         self.model.setScale(1, 1, 1)
#         self.model.setPos(0, 0, 5)
#         self.model.reparentTo(self.render)

#         # 创建 Bullet 物理形状
#         shape = BulletBoxShape((0.5, 0.5, 0.5))

#         # 创建 Bullet 物理节点
#         self.rigid_body_node = BulletRigidBodyNode('box')
#         self.rigid_body_node.addShape(shape)
#         self.rigid_body_node.setMass(1.0)  # 设置质量
#         self.rigid_body_np = self.render.attachNewNode(self.rigid_body_node)

#         # 将物理节点添加到物理世界中
#         self.bullet_world.attachRigidBody(self.rigid_body_node)

#         # 将模型和物理节点同步
#         self.model.reparentTo(self.rigid_body_np)

#         # 设置摄像机位置
#         self.camera.setPos(0, -10, 5)
#         self.camera.lookAt(self.render)

#     def update(self, task):
#         # 更新 Bullet 物理世界
#         dt = globalClock.getDt()
#         self.bullet_world.doPhysics(dt)
#         return task.cont

# app = MyApp()
# app.taskMgr.add(app.update, 'update')
# app.run()


In [1]:
import filepath

In [2]:
# TODO: a room with walls
# a camera that can fly
# from panda3d.core import PointLight, DirectionalLight
import filepath
from util.app import ContextShowBase
from util.log import Loggable
from direct.task import Task
import numpy as np
import torch
from geom.basic import create_cube_node, create_sphere_node,uv_curve_surface, create_colored_cube_node
from panda3d.core import (
    Geom,
    GeomNode,
    GeomTriangles,
    GeomVertexData,
    GeomVertexFormat,
    GeomVertexWriter,
    GeomEnums,
    NodePath,
    PointLight,
    DirectionalLight,
    CardMaker,
    WindowProperties
)
import ursina
from direct.showbase import DirectObject
from game.game_object import GameObject
from game.controller import PlayerController

# controller
# player controller
# agent controller
from typing import Set, List, Dict,Callable
from datetime import datetime
from abc import ABC
from direct.showbase.ShowBase import ShowBase
from panda3d.core import PNMImage, Texture
from panda3d.core import PTAUchar
from panda3d.core import CardMaker
from panda3d.core import Point2

import numpy as np
# import gizeh as gz
import random

In [3]:
class CameraController(ABC):
    # FIXME: use dict
    
    @classmethod
    def _move_forward(cls, task):
        raise NotImplementedError
    @classmethod
    def _move_backward(cls, task):
        raise NotImplementedError
    @classmethod
    def _turn_left(cls, task):
        raise NotImplementedError
    @classmethod
    def _turn_right(cls, task):
        raise NotImplementedError
        
    @classmethod
    def _move_up(cls, task):
        raise NotImplementedError
        
    @classmethod
    def _move_down(cls, task):
        raise NotImplementedError
    
    # TODO: check isinstance callable
    def set_move_forward(self, f):
        self._move_forward = f
    
    def set_move_backward(self,f):
        self._move_backward = f
        
    def set_turn_left(self, f):
        self._turn_left = f
        
    def set_turn_right(self, f):
        self._turn_right = f
        
    def set_move_up(self,f):
        self._move_up = f
        
    def set_move_down(self,f):
        self._move_down = f
        
    def call_move_forward(self, task):
        self._move_forward(task)
        
    def call_move_backward(self, task):
        self._move_backward(task)
        
    def call_turn_left(self, task):
        self._turn_left(task)
        
    def call_turn_right(self, task):
        self._turn_right(task)
        
    def call_move_down(self, task):
        self._move_down(task)
        
    def call_move_up(self, task):
        self._move_up(task)
        
class PlayerCubeController(CameraController, PlayerController):
    def __init__(self): # TODO: input keymap dict
        super().__init__()
        # from panda3d.core import KeyboardButton
        self.key_maps = {
            'w': self.call_move_forward,
            's': self.call_move_backward,
            'a': self.call_turn_left,
            'd': self.call_turn_right
            # up down
        }
        


In [4]:
from util.texture import (
    create_grey_checkerboard, 
    create_color_checkerboard,
    np2texture
)

In [5]:
# square_size = 512/32
# size = (512, 512+square_size)  # 图像尺寸 (宽度, 高度)
# h = int(size[0])
# w = int(size[1]) # FIXME
# # num_squares = 8    # 每行和每列的方格数量
# # checkerboard_arr = create_grey_checkerboard(size, square_size)
# checkerboard_arr = create_color_checkerboard(size, square_size)
# t = np2texture(
#     arr=checkerboard_arr,
#     format_=Texture.F_rgb,
#     name="checkerboard"
# )

In [6]:
def make_wall_texture(
    w, h,
    square_size:float=1,
    res:int=8,
    color1 = [0,0,0],
    color2 = [255,255,255],
    name = None
) -> Texture:
    w_img = int(w*res)
    h_img = int(h*res)
    square_size_img = int(square_size*res)
    checkerboard_arr = create_color_checkerboard(
        size=(h_img,w_img),
        square_size=square_size_img,
        color1=color1, color2=color2
    )[:,:,::-1]
    t = np2texture(
        arr=checkerboard_arr,
        format_=Texture.F_rgb,
        name=name
    )
    return t
    

In [29]:
class PhyscRoom(ContextShowBase):
    def __init__(self, xb:int, yb:int , zb:int):
        super().__init__()
        resolution = 8 # 8pxs a grid
        tex_top = make_wall_texture(
            yb*2,xb*2,square_size=5,
            res=resolution,
            color1 = [135,206,235] # celestial 
        )
        tex_bot = make_wall_texture(
            yb*2,xb*2,square_size=5,
            res=resolution,
            color1 = [227,189,101], # mustard
            color2 = [34,139,34] # grass
        )
        tex_xz = make_wall_texture(
            zb*2,xb*2,square_size=5,
            res=resolution,
            color1= [255,187,255] # plum
        )
        tex_yz = make_wall_texture(
            zb*2,yb*2,square_size=5,
            res=resolution,
            color1 = [0,173,131], # Pantone,
        )        
        
        # maka a room
        cardxy = CardMaker('wallxy')
        cardxy.setFrame(-xb,xb,-yb,yb)
        # FIXME
        cardxy.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))  # ul
        wall_xy_top = self.render.attach_new_node(cardxy.generate())
        wall_xy_top.set_pos(0,0,zb)
        wall_xy_top.setP(90)
        wall_xy_top.set_texture(tex_top)
        wall_xy_bot = self.render.attach_new_node(cardxy.generate())
        wall_xy_bot.set_pos(0,0,-zb)
        wall_xy_bot.setP(-90)
        wall_xy_bot.set_texture(tex_bot)
        cardxz = CardMaker('wallxz')
        cardxz.setFrame(-xb,xb,-zb,zb)
        cardxz.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))  
        wall_xz_front = self.render.attach_new_node(cardxz.generate())
        wall_xz_front.set_pos(0,yb,0)
        wall_xz_front.set_texture(tex_xz)
        wall_xz_back = self.render.attach_new_node(cardxz.generate())
        wall_xz_back.set_pos(0,-yb,0)
        wall_xz_back.setH(-180)
        wall_xz_back.set_texture(tex_xz)
        cardyz = CardMaker('wallyz')
        cardyz.setFrame(-yb,yb,-zb,zb)
        cardyz.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))  
        wall_yz_left = self.render.attach_new_node(cardyz.generate())
        wall_yz_left.set_pos(-xb,0,0)
        wall_yz_left.setH(90)
        wall_yz_left.set_texture(tex_yz)
        wall_yz_right = self.render.attach_new_node(cardyz.generate())
        wall_yz_right.set_pos(xb,0,0)
        wall_yz_right.setH(-90)
        wall_yz_right.set_texture(tex_yz)
        
        
        
        self.camera.setPos(5,5,10)
        self.accept('z', self.toggle_camera)
        # self.camera.setHpr(0,-90,0)
        
        
        sphere_o = create_sphere_node("o", 12, 12)
        self.render.attachNewNode(sphere_o)
        node_path_sphere_o = NodePath(sphere_o)
        node_path_sphere_o.setColor((1,1,1,1))
        node_path_sphere_o.setPos(0,0,0)
        sphere_x = create_sphere_node("x", 12, 12)
        self.render.attachNewNode(sphere_x)
        node_path_sphere_x = NodePath(sphere_x)
        node_path_sphere_x.setColor((1,0,0,1))
        node_path_sphere_x.setPos(1,0,0)
        sphere_y = create_sphere_node("y", 12, 12)
        self.render.attachNewNode(sphere_y)
        node_path_sphere_y = NodePath(sphere_y)
        node_path_sphere_y.setColor((0,1,0,1))
        node_path_sphere_y.setPos(0,1,0)
        sphere_z = create_sphere_node("z", 12, 12)
        self.render.attachNewNode(sphere_z)
        node_path_sphere_z = NodePath(sphere_z)
        node_path_sphere_z.setColor((0,0,1,1))
        node_path_sphere_z.setPos(0,0,1)
        node_path_sphere_o.setScale(0.2)
        node_path_sphere_x.setScale(0.2)
        node_path_sphere_y.setScale(0.2)
        node_path_sphere_z.setScale(0.2)
        
        # create a cube
        # self.player_cube =  ControlledCube("player")
        # self.player_cube.reparentTo(self.render)
        # self.player_cube.setZ(self.player_cube.scale/2)
        # self.player_cube.setRef(self.render)
        # self.controller = PlayerCubeController()
        # self.player_cube.register_controller(self.controller)
        self.buttonThrowers[0].node().setButtonDownEvent('button')
        self.buttonThrowers[0].node().setButtonUpEvent('button-up')
        # print("orient", self.player_cube.nodePath.getQuat(render))
        # orient 1 + 0i + 0j + 0k
        # print("orient", self.player_cube.nodePath.getQuat(render).getForward())
        # orient LVector3f(0, 1, 0)
     
    # 隐藏鼠标光标
        props = WindowProperties()
        props.setCursorHidden(True)
        self.win.requestProperties(props)

        # 将鼠标居中
        self.center_mouse()
        # 禁用默认的鼠标控制（这样摄像机就不会被默认控制）
        self.disableMouse()

        # 设定初始摄像机位置
        self.camera.setPos(0, -10, 1)

        # 每帧更新摄像机
        self.taskMgr.add(self.update_camera, "update_camera_task")
        # self.taskMgr.add(self.spin_card, "spin_card_task")
        # self.taskMgr.add(self.controller.update, "update_controller")

        # 保存鼠标的初始位置
        self.prev_mouse_x = 0
        self.prev_mouse_y = 0

    def center_mouse(self):
        """将鼠标指针重置到窗口的中心"""
        window_center_x = self.win.getXSize() // 2
        window_center_y = self.win.getYSize() // 2
        self.win.movePointer(0, window_center_x, window_center_y)
        self.prev_mouse_x = window_center_x
        self.prev_mouse_y = window_center_y
        
    def toggle_camera(self):
        self.camera.setHpr(0,0,0)

    def update_camera(self, task):
        """每帧更新摄像机的方向，使其跟随鼠标的移动"""
        if self.mouseWatcherNode.hasMouse():
            # 获取鼠标的位置（归一化的 -1 到 1 范围内）
            mouse_x = self.win.getPointer(0).getX()
            mouse_y = self.win.getPointer(0).getY()

            # 计算鼠标移动的增量
            delta_x = (mouse_x - self.prev_mouse_x)
            delta_y = mouse_y - self.prev_mouse_y

            # 调整摄像机的水平旋转和俯仰角度
            camera_h = self.camera.getH() - delta_x * 0.1
            camera_p = self.camera.getP() - delta_y * 0.1

            # 设置新的摄像机角度
            self.camera.setH(camera_h)
            self.camera.setP(camera_p)

            # 将鼠标指针重置到窗口的中心
            self.center_mouse()

        return task.cont
    
    def spin_card(self, task):
        time_ = task.time
        angleDegrees = time_ * 60.0

        # self.screen.setH(angleDegrees) # around Z
        # self.screen.setP(angleDegrees)  # around X
        return Task.cont
        

#     def spin_camera_task(self, task):
#         # time_ = task.time
#         time_=60
#         angleDegrees = time_ * 60.0
#         angleRadians = angleDegrees * (np.pi / 180.0)
#         self.camera.setPos(20 * np.sin(angleRadians), -20 * np.cos(angleRadians), 3)
#         self.camera.setHpr(angleDegrees, 0, 0)
#         # self.camera.setPos(1,1,3)
#         # self.camera.setHpr(90,0,0)

#         return Task.cont

    def toggle_fullscreen(self):
        props = WindowProperties()
        props.setFullscreen(not self.win.isFullscreen())
        self.win.requestProperties(props)
import builtins
import traceback
try:
    with PhyscRoom(5,25,10) as app:
        app.run()
except Exception as e:
    print(e)
    print(traceback.format_exc())
finally:
    if hasattr(builtins, 'base'):
        builtins.base.destroy()              

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


---<__main__.PhyscRoom object at 0x7f7149c0d150> run(), at 2024-08-25 22:55:33.914984---
---<__main__.PhyscRoom object at 0x7f7149c0d150> destroy at 2024-08-25 22:55:36.589698---
---<__main__.PhyscRoom object at 0x7f7149c0d150> destroyed at 2024-08-25 22:55:36.590827, exit---


SystemExit: 

In [19]:
0.87*255

221.85